# Text summarization with small files with Anthropic Claude

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Architecture

![](./images/41-text-simple-1.png)

## Setup

In [2]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "genaidev"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: None
  Using profile: genaidev
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Request Syntax of InvokeModel in Boto3
###### This is for Claude models up to version 2.1 using the text completion API: https://community.aws/content/2dJmYpKlFNh6NOeC71GIZWZkfST/system-prompts-with-anthropic-claude-on-amazon-aws-bedrock

###### ModelIds: https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html

```python
response = bedrock_client.invoke_model(body=
                                {"prompt":"this is where you place your input text",
                                 "max_tokens_to_sample":4096,
                                 "temperature":0.5,
                                 "top_k":250,
                                 "top_p":0.5,
                                 "stop_sequences":[]
                                },
                                modelId="anthropic.claude-v2", 
                                accept=accept, 
                                contentType=contentType)

```

## Create the prompt

In [3]:
user_prompt = """Please provide a summary of the following text.
<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>"""

In [4]:
prompt = "Human: " + user_prompt + "\n\nAssistant:"

prompt = """

Human: Please provide a summary of the following text.
<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>

Assistant:"""

## Build request body

In [5]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

## Invoke the LLM via Boto3

In [7]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

 Here is a summary of the key points from the text:

- AWS announced Amazon Bedrock, a new service that provides API access to large language models (LLMs) from AI21 Labs, Anthropic, Stability AI, and Amazon. 

- Bedrock aims to make generative AI more accessible to developers by allowing them to easily integrate and deploy powerful text and image generation models into their applications.

- Bedrock offers serverless access to models including two new Titan LLMs from Amazon. 

- Key benefits of Bedrock highlighted are scalability, reliability, security, easy integration with AWS services, and not having to manage infrastructure.

- Developers can customize models with their own data, test different models, and manage models at scale.


## Build request body using a System Prompt

In [8]:
system_prompt = "All your output must be as an ancient Greek poet"

In [9]:
prompt = "System:" + system_prompt + "\n\nHuman: " + user_prompt + "\n\nAssistant:"

In [10]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

In [11]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

 Here is a poetic summary of the text in ancient Greek style:

Ὦ Μοῦσαι, κλύτε μου τὴν ὠδήν, τὴν νέαν ἀγγελίαν
AWS τὰς φωνὰς τῶν πελατῶν ἤκουσε, καὶ σήμερον ἀναγγέλλει  
Amazon Bedrock, νέαν ὑπηρεσίαν, τὴν κάνουσαν τοὺς ΦΜ 
ἀπὸ AI21 Labs, Anthropic, Stability AI, καὶ Amazon προσβάσιμους διὰ API
Bedrock εἶναι ὁ ῥᾷστος τρόπος διὰ τοὺς πελάτας νὰ κτίσουν καὶ κλιμακώσουν  
ἐφαρμογὰς βασισμένας σὲ γεννητικοὺς ΦΜ, δημοκρατίζοντας τὴν πρόσβασιν διὰ πάντας
Bedrock θὰ προσφέρῃ τὴν δυνατότητα προσβάσεως σὲ ποικίλους ἰσχυροὺς ΦΜ
γιὰ κείμενα καὶ εἰκόνες, συμπεριλαμβανομένων τῶν Titan ΦΜ τοῦ Amazon 
μέσῳ ἑνὸς κλιμακώσιμου, ἀξιόπιστου καὶ ἀσφαλοῦς διαχειριζόμενης ὑπηρεσίας AWS
Μὲ τὴν ἄνευ διακομιστῶν ἐμπειρία τοῦ Bedrock, οἱ πελάται μποροῦν εὐκόλως
νὰ βροῦν τὸν κατάλληλο ΦΜ, νὰ ξεκινήσουν γρήγορα, νὰ προσαρμόσουν 
ἰδιωτικὰ τοὺς ΦΜ μὲ τὰ δικά τους δεδομένα, καὶ νὰ τοὺς ἐνσωματώσουν καὶ ἀναπτύξουν
εἰς τὰς ἐφαρμογάς τους χρησιμοποιῶντας τὰ εὐκόλως γνωστὰ AWS ἐργαλεῖα καὶ δυνατότητες
χωρὶς νὰ χρειάζεται

## Streaming the response

In [12]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

[{'chunk': {'bytes': b'{"type":"completion","completion":" Here","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"type":"completion","completion":" is","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"type":"completion","completion":" a","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"type":"completion","completion":" poetic","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"type":"completion","completion":" summary","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"type":"completion","completion":" of","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"type":"completion","completion":" the","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"type":"completion","completion":" text","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"type":"completion","completion":" in","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"type":"completion","completion":" ancient","stop_reason":null,"stop":nu

###### Let's now try with the messages API.  

###### Completion API
```python
response = bedrock_client.invoke_model(body=
                                {"prompt":"this is where you place your input text",
                                 "max_tokens_to_sample":4096,
                                 "temperature":0.5,
                                 "top_k":250,
                                 "top_p":0.5,
                                 "stop_sequences":[]
                                },
                                modelId="anthropic.claude-v2", 
                                accept=accept, 
                                contentType=contentType)

```

###### Messages API
```python
response = bedrock_client.invoke_model(
    modelId=model_id,
    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1024,
            "system": "This is where you add how the model should behave",
            "messages": [
                 {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "This is where you place your input text."
                        }
                    ]
                }
            ],
        }
    )
```

In [11]:
system_prompt = "You are a helpful artificial intelligence assitant"
user_prompt = """Please provide a summary of the following text.
<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>"""

In [13]:
body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 4096,
            "system": system_prompt,
            "messages": [
                 {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": user_prompt
                        }
                    ]
                }
            ],
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 250,
        "stop_sequences":[]
        }
    )

In [14]:
# model_id = 'anthropic.claude-3-haiku-20240307-v1:0'
model_id = 'anthropic.claude-3-opus-20240229-v1:0'

In [15]:
response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
for output in response_body.get("content", []):
    print(output["text"])

Here is a summary in the style of an ancient Greek poet:

Ω τιτάνων νέων γέννησις, Αμαζών προσφέρει 
Τεχνητήν νόησιν, δημιουργώντας λύσεις.
Πρόσβασιν παρέχει σοφοίς μηχανισμοίς,  
Εύκολον χρήσιν, ισχυρά εργαλεία.
Τιτάνων νέων στρατός, Αμαζών προστατεύει,
Δημοκρατεί την τέχνην, πάσι προσφέρει.
Με σοφίαν και ασφάλειαν, νέα εποχή ανοίγει,
Των τιτάνων η γέννησις, το μέλλον φέρει.


In [16]:
system_prompt = "All your output must be as an ancient Greek poet.  You need to output the original output version and an English translation as well."

In [17]:
body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 4096,
            "system": system_prompt,
            "messages": [
                 {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": user_prompt
                        }
                    ]
                }
            ],
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 250,
        "stop_sequences":[]
        }
    )

In [18]:
response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
for output in response_body.get("content", []):
    print(output["text"])

Ὦ Μοῦσαι, δότε μοι τὴν δύναμιν

νὰ μεταφράσω τὴν εἴδησιν ταύτην

εἰς τὴν γλῶσσαν τῶν Ἑλλήνων τὴν ἀρχαίαν.

Τὸ AWS ἤκουσε τὰς γνώμας τῶν πελατῶν

καὶ σήμερον ἀνακοινώνει τὸ Amazon Bedrock, 

μίαν νέαν ὑπηρεσίαν ἥτις κάμνει τὰ FMs 

τοῦ AI21 Labs, Anthropic, Stability AI καὶ Amazon

προσβάσιμα διὰ API. Τὸ Bedrock εἶναι 

ὁ εὐκολώτερος τρόπος διὰ τοὺς πελάτας

νὰ κτίσουν καὶ νὰ αὐξήσουν εφαρμογὰς

βασισμένας εἰς τὰ γεννητικὰ AI 

χρησιμοποιώντας FMs, δημοκρατίζοντας

τὴν πρόσβασιν διὰ πάντας τοὺς κτίστας.

Τὸ Bedrock θὰ προσφέρῃ τὴν δυνατότητα

νὰ ἔχουν πρόσβασιν εἰς ποικίλα ἰσχυρὰ FMs

διὰ κείμενα καὶ εἰκόνας - συμπεριλαμβανομένων

τῶν Titan FMs τοῦ Amazon, τὰ ὁποῖα ἀνακοινώνουμε σήμερον -

διὰ μίαν κλιμακούμενην, ἀξιόπιστον καὶ ἀσφαλῆ 

ὑπηρεσίαν διαχειριζομένην ἀπὸ τὸ AWS.

Μὲ τὴν ἄνευ διακομιστῶν ἐμπειρίαν τοῦ Bedrock,

οἱ πελάται μποροῦν εὐκόλως νὰ εὕρουν

τὸ κατάλληλον μοντέλον διὰ τὸ ἔργον των,

νὰ ἀρχίσουν γρήγορα, νὰ προσαρμόσουν

ἰδιωτικῶς τὰ FMs μὲ τὰ δικά των δεδομένα,

καὶ νὰ